# Data

In [ ]:
import abc
from heapq import heappush, heappop
import collections
from itertools import combinations, islice
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Lectura del Dataset
dscorona = pd.read_excel('/content/covid_DB.xlsx')
dscorona

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,ae66feb9e4dc3a0,3,positive,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5640,517c2834024f3ea,17,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,5c57d6037fe266d,4,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5642,c20c44766f28291,10,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dscorona.shape #Dimensiones de dataset

(5644, 111)

In [ ]:
# número de Columnas
for i in range(dscorona.shape[1]):
    print(dscorona.columns[i])


Patient ID
Patient age quantile
SARS-Cov-2 exam result
Patient addmited to regular ward (1=yes, 0=no)
Patient addmited to semi-intensive unit (1=yes, 0=no)
Patient addmited to intensive care unit (1=yes, 0=no)
Hematocrit
Hemoglobin
Platelets
Mean platelet volume 
Red blood Cells
Lymphocytes
Mean corpuscular hemoglobin concentration (MCHC)
Leukocytes
Basophils
Mean corpuscular hemoglobin (MCH)
Eosinophils
Mean corpuscular volume (MCV)
Monocytes
Red blood cell distribution width (RDW)
Serum Glucose
Respiratory Syncytial Virus
Influenza A
Influenza B
Parainfluenza 1
CoronavirusNL63
Rhinovirus/Enterovirus
Mycoplasma pneumoniae
Coronavirus HKU1
Parainfluenza 3
Chlamydophila pneumoniae
Adenovirus
Parainfluenza 4
Coronavirus229E
CoronavirusOC43
Inf A H1N1 2009
Bordetella pertussis
Metapneumovirus
Parainfluenza 2
Neutrophils
Urea
Proteina C reativa mg/dL
Creatinine
Potassium
Sodium
Influenza B, rapid test
Influenza A, rapid test
Alanine transaminase
Aspartate transaminase
Gamma-glutamyltransfe

In [ ]:
# descripción del dataset
dscorona.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# resumen corto de las variables cuantitativas
dscorona.describe()

,Patient age quantile,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
count,5644.000000,5644.000000,5644.000000,5644.000000,6.030000e+02,6.030000e+02,6.020000e+02,5.990000e+02,6.020000e+02,6.020000e+02,...,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.000000e+01,2.000000e+01,2.700000e+01
mean,9.318391,0.013997,0.008859,0.007264,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438142e-09,8.424447e-09,-7.866736e-09,...,-1.379737e-10,8.416397e-09,-1.655685e-09,4.139211e-10,-7.472139e-09,6.070843e-09,-2.469729e-08,4.656613e-09,6.332993e-09,5.243001e-09
std,5.777903,0.117489,0.093713,0.084929,1.000830e+00,1.000830e+00,1.000832e+00,1.000836e+00,1.000832e+00,1.000832e+00,...,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.025978e+00,1.025978e+00,1.019049e+00
min,0.000000,0.000000,0.000000,0.000000,-4.501420e+00,-4.345603e+00,-2.552426e+00,-2.457575e+00,-3.970608e+00,-1.865070e+00,...,-1.999560e+00,-1.244817e+00,-3.082674e+00,-3.568877e+00,-2.925618e+00,-2.985592e+00,-1.175907e+00,-1.532932e+00,-1.480526e+00,-2.900254e+00
25%,4.000000,0.000000,0.000000,0.000000,-5.188074e-01,-5.862439e-01,-6.053457e-01,-6.624832e-01,-5.679496e-01,-7.307069e-01,...,-1.122574e+00,-5.348102e-01,-3.308668e-01,-9.210583e-02,-5.117720e-01,-5.397211e-01,-8.169898e-01,-1.214975e-01,-5.527296e-01,-4.852787e-01
50%,9.000000,0.000000,0.000000,0.000000,5.340703e-02,4.031596e-02,-1.217160e-01,-1.015171e-01,1.385207e-02,-1.426696e-02,...,2.677689e-01,-2.120799e-01,-1.181667e-02,2.942021e-01,7.743482e-02,5.633191e-02,-1.599549e-01,-1.174366e-02,-1.381825e-01,1.826928e-01
75%,14.000000,0.000000,0.000000,0.000000,7.171751e-01,7.295320e-01,5.314981e-01,6.838353e-01,6.661759e-01,5.976919e-01,...,7.383496e-01,2.305214e-02,6.661649e-01,5.115003e-01,4.385609e-01,5.085100e-01,4.500090e-01,-1.174366e-02,2.763648e-01,5.937525e-01
max,19.000000,1.000000,1.000000,1.000000,2.662704e+00,2.671868e+00,9.532034e+00,3.713052e+00,3.645706e+00,3.764100e+00,...,1.337265e+00,3.236524e+00,1.703078e+00,1.042674e+00,1.940087e+00,2.029471e+00,2.205371e+00,2.841856e+00,2.862350e+00,1.826932e+00


In [ ]:
dscorona.describe(include='all')

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
count,5644,5644.000000,5644,5644.000000,5644.000000,5644.000000,6.030000e+02,6.030000e+02,6.020000e+02,5.990000e+02,...,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.700000e+01,2.000000e+01,2.000000e+01,2.700000e+01
unique,5644,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,44477f75e8169d2,NaN,negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,5086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,9.318391,NaN,0.013997,0.008859,0.007264,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438142e-09,...,-1.379737e-10,8.416397e-09,-1.655685e-09,4.139211e-10,-7.472139e-09,6.070843e-09,-2.469729e-08,4.656613e-09,6.332993e-09,5.243001e-09
std,NaN,5.777903,NaN,0.117489,0.093713,0.084929,1.000830e+00,1.000830e+00,1.000832e+00,1.000836e+00,...,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.019049e+00,1.025978e+00,1.025978e+00,1.019049e+00
min,NaN,0.000000,NaN,0.000000,0.000000,0.000000,-4.501420e+00,-4.345603e+00,-2.552426e+00,-2.457575e+00,...,-1.999560e+00,-1.244817e+00,-3.082674e+00,-3.568877e+00,-2.925618e+00,-2.985592e+00,-1.175907e+00,-1.532932e+00,-1.480526e+00,-2.900254e+00
25%,NaN,4.000000,NaN,0.000000,0.000000,0.000000,-5.188074e-01,-5.862439e-01,-6.053457e-01,-6.624832e-01,...,-1.122574e+00,-5.348102e-01,-3.308668e-01,-9.210583e-02,-5.117720e-01,-5.397211e-01,-8.169898e-01,-1.214975e-01,-5.527296e-01,-4.852787e-01
50%,NaN,9.000000,NaN,0.000000,0.000000,0.000000,5.340703e-02,4.031596e-02,-1.217160e-01,-1.015171e-01,...,2.677689e-01,-2.120799e-01,-1.181667e-02,2.942021e-01,7.743482e-02,5.633191e-02,-1.599549e-01,-1.174366e-02,-1.381825e-01,1.826928e-01
75%,NaN,14.000000,NaN,0.000000,0.000000,0.000000,7.171751e-01,7.295320e-01,5.314981e-01,6.838353e-01,...,7.383496e-01,2.305214e-02,6.661649e-01,5.115003e-01,4.385609e-01,5.085100e-01,4.500090e-01,-1.174366e-02,2.763648e-01,5.937525e-01


In [ ]:
print('Número de filas :',dscorona.shape[0])
print('Número de columnas :',dscorona.shape[1])

Número de filas : 5644
Número de columnas : 111


# Preprocesar data

In [ ]:
# Muestra los primeros 20 nombres de las columnas
primeros_20_nombres = dscorona.columns[:20]
print(primeros_20_nombres)

Index(['Patient ID', 'Patient age quantile', 'SARS-Cov-2 exam result',
       'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'Hematocrit',
       'Hemoglobin', 'Platelets', 'Mean platelet volume ', 'Red blood Cells',
       'Lymphocytes', 'Mean corpuscular hemoglobin concentration (MCHC)',
       'Leukocytes', 'Basophils', 'Mean corpuscular hemoglobin (MCH)',
       'Eosinophils', 'Mean corpuscular volume (MCV)', 'Monocytes',
       'Red blood cell distribution width (RDW)'],
      dtype='object')


In [ ]:
columnas_seleccionadas = ["SARS-Cov-2 exam result", "Patient age quantile", "Hematocrit", "Platelets", 'Mean platelet volume ', 'Mean corpuscular hemoglobin (MCH)', "Leukocytes", "Basophils", "Eosinophils", "Monocytes", "Proteina C reativa mg/dL"]

# Seleccionar solo las columnas deseadas
subset_dscorona = dscorona[columnas_seleccionadas]

# Reemplazar NaN con 0
subset_dscorona.fillna(0, inplace=True)

# Mapear los valores de la columna "SARS-Cov-2 exam result" a 1 para "positive" y 0 para "negative"
subset_dscorona["SARS-Cov-2 exam result"] = subset_dscorona["SARS-Cov-2 exam result"].map({'positive': 1, 'negative': 0})

# Convertir la columna "SARS-Cov-2 exam result" a tipo float
subset_dscorona["SARS-Cov-2 exam result"] = subset_dscorona["SARS-Cov-2 exam result"].astype(float)

# Convertir la columna "Patient age quantile" a tipo float
subset_dscorona["Patient age quantile"] = subset_dscorona["Patient age quantile"].astype(float)

# Imprimir la información sobre los tipos de datos
tipos_de_datos = subset_dscorona.dtypes
print(tipos_de_datos)


SARS-Cov-2 exam result               float64
Patient age quantile                 float64
Hematocrit                           float64
Platelets                            float64
Mean platelet volume                 float64
Mean corpuscular hemoglobin (MCH)    float64
Leukocytes                           float64
Basophils                            float64
Eosinophils                          float64
Monocytes                            float64
Proteina C reativa mg/dL             float64
dtype: object


<ipython-input-11-9b7c4399a803>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dscorona.fillna(0, inplace=True)
<ipython-input-11-9b7c4399a803>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dscorona["SARS-Cov-2 exam result"] = subset_dscorona["SARS-Cov-2 exam result"].map({'positive': 1, 'negative': 0})
<ipython-input-11-9b7c4399a803>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
subset_dscorona

,SARS-Cov-2 exam result,Patient age quantile,Hematocrit,Platelets,Mean platelet volume,Mean corpuscular hemoglobin (MCH),Leukocytes,Basophils,Eosinophils,Monocytes,Proteina C reativa mg/dL
0,0.0,13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,17.0,0.236515,-0.517413,0.010677,-0.292269,-0.094610,-0.223767,1.482158,0.357547,-0.147895
2,0.0,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5639,1.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5640,0.0,17.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5641,0.0,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5642,0.0,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# M-Tree

In [ ]:
import collections
import abc
from heapq import heappush
from itertools import islice, combinations

class MTree:
    def __init__(self, max_node_size, distance_function):
        self.max_node_size = max_node_size
        self.distance_function = distance_function
        self.root = None

    def promote(self, entries, current_routing_entry, d):
        if current_routing_entry is None or \
                any(e.distance_to_parent is None for e in entries):
            return self.promote_non_confirmed(entries, current_routing_entry, d)

        new_entry = max(entries, key=lambda e: e.distance_to_parent)
        return (current_routing_entry.obj, new_entry.obj)

    def promote_non_confirmed(self, entries, unused_current_routing_entry, d):
        objs = map(lambda e: e.obj, entries)
        return max(combinations(objs, 2), key=lambda two_objs: d(*two_objs))

    def partition(self, entries, routing_object1, routing_object2, d):
        partition = (set(), set())
        for entry in entries:
            partition[d(entry.obj, routing_object1) > \
                         d(entry.obj, routing_object2)].add(entry)

        if not partition[0] or not partition[1]:
            partition = (set(islice(entries, len(entries)//2)),
                         set(islice(entries, len(entries)//2, len(entries))))

        return partition

    def insert(self, obj):
        if self.root is None:
            self.root = LeafNode(self, entries={Entry(obj)})
        else:
            self.root.add(obj)

    def range_query(self, query_obj, radius):
        results = []

        def recursive_range_search(node):
            if isinstance(node, LeafNode):
                for entry in node.entries:
                    distance = mtree.distance_function(entry.obj, query_obj)
                    if distance <= radius:
                        results.append(entry.obj)
            else:
                for entry in node.entries:
                    dist_to_obj = mtree.distance_function(entry.obj, query_obj)
                    if dist_to_obj <= entry.radius + radius:
                        recursive_range_search(entry.subtree)

        recursive_range_search(self.root)
        return results

    def nearest_neighbors(self, query_obj, k):
        nn = NN(size=k)
        pr = []
        d_parent_query = float("inf")
        self.root.search(query_obj, pr, nn, d_parent_query)
        return nn.result_list()

def M_LB_DIST_confirmed(entries, current_routing_entry, d):
    if current_routing_entry is None or any(e.distance_to_parent is None for e in entries):
        return M_LB_DIST_non_confirmed(entries, current_routing_entry, d)

    new_entry = max(entries, key=lambda e: e.distance_to_parent)
    return (current_routing_entry.obj, new_entry.obj)

def M_LB_DIST_non_confirmed(entries, unused_current_routing_entry, d):
    objs = map(lambda e: e.obj, entries)
    return max(combinations(objs, 2), key=lambda two_objs: d(*two_objs))

def generalized_hyperplane(entries, routing_object1, routing_object2, d):
    partition = (set(), set())
    for entry in entries:
        partition[d(entry.obj, routing_object1) > d(entry.obj, routing_object2)].add(entry)

    if not partition[0] or not partition[1]:
        partition = (set(islice(entries, len(entries)//2)), set(islice(entries, len(entries)//2, len(entries))))

    return partition

NNEntry = collections.namedtuple('NNEntry', 'obj dmax')

class NN(object):
    def __init__(self, size):
        self.elems = [NNEntry(None, float("inf"))] * size
        self.dmax = float("inf")

    def __len__(self):
        return len(self.elems)

    def search_radius(self):
        return self.dmax

    def update(self, obj, dmax):
        if obj is None:
            self.dmax = min(self.dmax, dmax)
            return
        self.elems.append(NNEntry(obj, dmax))
        for i in range(len(self) - 1, 0, -1):
            if self.elems[i].dmax < self.elems[i-1].dmax:
                self.elems[i-1], self.elems[i] = self.elems[i], self.elems[i-1]
            else:
                break
        self.elems.pop()

    def result_list(self):
        result = map(lambda entry: entry.obj, self.elems)
        return result

    def __repr__(self):
        return "NN(%r)" % self.elems

class PrEntry(object):
    def __init__(self, tree, dmin, d_query):
        self.tree = tree
        self.dmin = dmin
        self.d_query = d_query

    def __lt__(self, other):
        return self.dmin < other.dmin

    def __repr__(self):
        return "PrEntry(tree:%r, dmin:%r)" % (self.tree, self.dmin)

class Entry(object):
    def __init__(self, obj, distance_to_parent=None, radius=None, subtree=None):
        self.obj = obj
        self.distance_to_parent = distance_to_parent
        self.radius = radius
        self.subtree = subtree

    def __repr__(self):
        return "Entry(obj: %r, dist: %r, radius: %r, subtree: %r)" % (
            self.obj,
            self.distance_to_parent,
            self.radius,
            self.subtree.repr_class() if self.subtree else self.subtree)

class AbstractNode(object):
    __metaclass__ = abc.ABCMeta

    def __init__(self, mtree, parent_node=None, parent_entry=None, entries=None):
        self.mtree = mtree
        self.parent_node = parent_node
        self.parent_entry = parent_entry
        self.entries = set(entries) if entries else set()

    def __repr__(self):
        entries_str = '%s' % list(islice(self.entries, 2))
        if len(self.entries) > 2:
            entries_str = entries_str[:-1] + ', ...]'
        return "%s(parent_node: %s, parent_entry: %s, entries:%s)" % (
            self.__class__.__name__,
            self.parent_node.repr_class() if self.parent_node else self.parent_node,
            self.parent_entry,
            entries_str
        )

    def repr_class(self):  # pragma: no cover
        return self.__class__.__name__ + "()"

    def __len__(self):
        return len(self.entries)


    @property
    def d(self):
        return self.mtree.distance_function

    def is_full(self):
        return len(self) == self.mtree.max_node_size

    def is_empty(self):
        return len(self) == 0

    def is_root(self):
        return self is self.mtree.root

    def remove_entry(self, entry):
        self.entries.remove(entry)

    def add_entry(self, entry):
        if self.is_full():
            raise ValueError('Trying to add %s into a full node' % str(entry))
        self.entries.add(entry)

    def set_entries_and_parent_entry(self, new_entries, new_parent_entry):
        self.entries = new_entries
        self.parent_entry = new_parent_entry
        self.parent_entry.radius = self.covering_radius_for(self.parent_entry.obj)
        self._update_entries_distance_to_parent()

    def _update_entries_distance_to_parent(self):
        if self.parent_entry:
            for entry in self.entries:
                entry.distance_to_parent = self.d(entry.obj, self.parent_entry.obj)

    @abc.abstractmethod
    def add(self, obj):  # pragma: no cover
        """Add obj into this subtree"""
        pass

    @abc.abstractmethod
    def covering_radius_for(self, obj):  # pragma: no cover
        """Compute the radius needed for obj to cover the entries of this node."""
        pass

    @abc.abstractmethod
    def search(self, query_obj, pr, nn, d_parent_query):
        pass

class LeafNode(AbstractNode):
    """A leaf of the M-tree"""
    def __init__(self, mtree, parent_node=None, parent_entry=None, entries=None):
        AbstractNode.__init__(self, mtree, parent_node, parent_entry, entries)

    def add(self, obj):
        distance_to_parent = self.d(obj, self.parent_entry.obj) if self.parent_entry else None
        new_entry = Entry(obj, distance_to_parent)
        if not self.is_full():
            self.entries.add(new_entry)
        else:
            split(self, new_entry, self.d)
        assert self.is_root() or self.parent_node

    def covering_radius_for(self, obj):
        if not self.entries:
            return 0
        else:
            return max(map(lambda e: self.d(obj, e.obj), self.entries))

    def could_contain_results(self, query_obj, search_radius, distance_to_parent, d_parent_query):
        if self.is_root():
            return True
        return abs(d_parent_query - distance_to_parent) <= search_radius

    def search(self, query_obj, pr, nn, d_parent_query):
        for entry in self.entries:
            if self.could_contain_results(query_obj, nn.search_radius(), entry.distance_to_parent, d_parent_query):
                distance_entry_to_q = self.d(entry.obj, query_obj)
                if distance_entry_to_q <= nn.search_radius():
                    nn.update(entry.obj, distance_entry_to_q)

class InternalNode(AbstractNode):
    def __init__(self, mtree, parent_node=None, parent_entry=None, entries=None):
        AbstractNode.__init__(self, mtree, parent_node, parent_entry, entries)

    def add(self, obj):
        dist_to_obj = {}
        for entry in self.entries:
            dist_to_obj[entry] = self.d(obj, entry.obj)

        def find_best_entry_requiring_no_covering_radius_increase():
            valid_entries = [e for e in self.entries if dist_to_obj[e] <= e.radius]
            return min(valid_entries, key=dist_to_obj.get) if valid_entries else None

        def find_best_entry_minimizing_radius_increase():
            entry = min(self.entries, key=lambda e: dist_to_obj[e] - e.radius)
            entry.radius = dist_to_obj[entry]
            return entry

        entry = find_best_entry_requiring_no_covering_radius_increase() or find_best_entry_minimizing_radius_increase()
        entry.subtree.add(obj)
        assert self.is_root() or self.parent_node

    def covering_radius_for(self, obj):
        if not self.entries:
            return 0
        else:
            return max(map(lambda e: self.d(obj, e.obj) + e.radius, self.entries))

    def set_entries_and_parent_entry(self, new_entries, new_parent_entry):
        AbstractNode.set_entries_and_parent_entry(self, new_entries, new_parent_entry)
        for entry in self.entries:
            entry.subtree.parent_node = self

    def could_contain_results(self, query_obj, search_radius, entry, d_parent_query):
        if self.is_root():
            return True
        parent_obj = self.parent_entry.obj
        return abs(d_parent_query - entry.distance_to_parent) <= search_radius + entry.radius

    def search(self, query_obj, pr, nn, d_parent_query):
        for entry in self.entries:
            if self.could_contain_results(query_obj, nn.search_radius(), entry, d_parent_query):
                d_entry_query = self.d(entry.obj, query_obj)
                entry_dmin = max(d_entry_query - entry.radius, 0)
                if entry_dmin <= nn.search_radius():
                    heappush(pr, PrEntry(entry.subtree, entry_dmin, d_entry_query))
                    entry_dmax = d_entry_query + entry.radius
                    if entry_dmax < nn.search_radius():
                        nn.update(None, entry_dmax)

def split(existing_node, entry, d):
    assert existing_node.is_full()
    mtree = existing_node.mtree

    new_node = type(existing_node)(existing_node.mtree)
    all_entries = existing_node.entries | set((entry,))

    routing_object1, routing_object2 = mtree.promote(all_entries, existing_node.parent_entry, d)
    entries1, entries2 = mtree.partition(all_entries, routing_object1, routing_object2, d)
    assert entries1 and entries2, "Error during split operation. All the entries have been assigned to one routing_objects and none to the other! Should never happen since at least the routing objects are assigned to their corresponding set of entries"

    old_existing_node_parent_entry = existing_node.parent_entry
    existing_node_entry = Entry(routing_object1, None, None, existing_node)
    existing_node.set_entries_and_parent_entry(entries1, existing_node_entry)

    new_node_entry = Entry(routing_object2, None, None, new_node)
    new_node.set_entries_and_parent_entry(entries2, new_node_entry)

    if existing_node.is_root():
        new_root_node = InternalNode(existing_node.mtree)
        existing_node.parent_node = new_root_node
        new_root_node.add_entry(existing_node_entry)
        new_node.parent_node = new_root_node
        new_root_node.add_entry(new_node_entry)
        mtree.root = new_root_node
    else:
        parent_node = existing_node.parent_node
        if not parent_node.is_root():
            existing_node_entry.distance_to_parent = d(existing_node_entry.obj, parent_node.parent_entry.obj)
            new_node_entry.distance_to_parent = d(new_node_entry.obj, parent_node.parent_entry.obj)

        parent_node.remove_entry(old_existing_node_parent_entry)
        parent_node.add_entry(existing_node_entry)

        if parent_node.is_full():
            split(parent_node, new_node_entry, d)
        else:
            parent_node.add_entry(new_node_entry)
            new_node.parent_node = parent_node
    assert existing_node.is_root() or existing_node.parent_node
    assert new_node.is_root() or new_node.parent_node

# Creacion del arbol como objeto

In [ ]:
# Supongamos que tienes una función de distancia Euclidiana para tus datos de 11 dimensiones
def euclidean_distance(point1, point2):
    return sum((float(x) - float(y))**2 for x, y in zip(point1, point2))**0.5

# Crea un árbol M con una capacidad máxima de nodos de 4 y la función de distancia Euclidiana
mtree = MTree(max_node_size=4, distance_function=euclidean_distance)

# Convertir las filas de subset_dscorona a tuplas
data_points_subset = [tuple(row) for _, row in subset_dscorona.iterrows()]

# Iterar sobre las filas y insertar en el árbol M
for point in data_points_subset:
    print("Insertar: ", point)
    mtree.insert(point)

Se han truncado las últimas 5000 líneas del flujo de salida.
Insertar:  (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 19.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Insertar:  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

# Consultas

In [ ]:
# Realiza una consulta de rango
query_point = (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
radius = 3
range_results = mtree.range_query(query_point, radius)
print("Resultados de la consulta de rango:", range_results)

Resultados de la consulta de rango: [(0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 14.0, 0.2594036757946014, 0.902071475982666, -0.1015170812606812, -0.8149853944778442, 0.1502753645181656, 0.6926107406616211, 0.2179767936468124, -0.03640146553516388, -0.447933703660965), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 14.0, 0

In [ ]:
# Encuentra los 1 vecinos más cercanos
query_point = (0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
k_neighbors = 4
nn_results = mtree.nearest_neighbors(query_point, k_neighbors)

# Convierte el objeto map a una lista
nn_results_list = list(nn_results)

print(f"{k_neighbors} vecinos más cercanos:", nn_results_list)


4 vecinos más cercanos: [None, None, None, None]
